In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Activation,BatchNormalization,Lambda,Input,Conv2D,Bidirectional,LSTM, Concatenate, MaxPooling2D,GlobalAveragePooling2D
import tensorflow.keras.backend as K
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from time import time
import datetime
import numpy as np
from generater import crnnGenerator
import logging
logger=tf.get_logger()
logger.setLevel(logging.ERROR)


In [1]:
import keras
keras.__version__

'2.5.0'

In [2]:
keras.optimizers.A
def convert_to_onehot(data):
    #Creates a dict, that maps to every char of alphabet an unique int based on position
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
    char_to_int = dict((c,i) for i,c in enumerate(alphabet))
    
    encoded_data = []
    #Replaces every char in data with the mapped int
    encoded_data.append([char_to_int[char] for char in data])
    
    

    #This part now replaces the int by an one-hot array with size alphabet
    one_hot = []
    for value in encoded_data:
        #At first, the whole array is initialized with 0
        
        for indexvalue in value:
            letter = [0 for _ in range(len(alphabet))]
            #Only at the number of the int, 1 is written

            letter[indexvalue] = 1
            
            one_hot.append(letter)
    return one_hot
import string
def convert_to_lable(data):
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
    alphabet87 = string.ascii_lowercase + string.ascii_uppercase + string.digits + ' +-*.,:!?%&$~/()[]<>"\'@#_'
    return list(map(lambda x: alphabet87.index(x), data))



convert_to_lable('abcd453')


[0, 1, 2, 3, 56, 57, 55]

In [6]:

def crnn():
    x=image_input=Input(name='image_input',shape=(256,32,1))
    """
    minx=Input(name='minx',shape=[1],dtype='float32')
    miny=Input(name='miny',shape=[1],dtype='float32')
    maxx=Input(name='maxx',shape=[1],dtype='float32')
    maxy=Input(name='maxy',shape=[1],dtype='float32')
    def cropimage(img,minx=0,miny=0,maxx=0,maxy=0):
        
        return tf.image.crop_to_bounding_box(img,minx,miny,maxx-minx,maxy-miny)
    x=Lambda(cropimage)([x,minx,miny,maxx,maxy])
    
    """
        
    x=Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv1_1')(x)#128 64 64
    x=MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1', padding='same')(x)#64 32 64
    
    x=Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv2_1')(x)#64 32 128
    x=MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool2', padding='same')(x)# 32 16 128

    x=Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_1')(x)#32 16 256
    x=Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_2')(x)#32 16 256
    x=MaxPooling2D(pool_size=(2, 2), strides=(1, 2), name='pool3', padding='same')(x)# 32 8 256
    
    x=Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv4_1')(x)#32 8 512
    x=BatchNormalization(name='batchnorm1')(x)
    
    x=Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv5_1')(x)#32 8 512
    
    x=BatchNormalization(name='batchnorm2')(x)
    x=MaxPooling2D(pool_size=(2, 2), strides=(1, 2),padding='valid', name='pool5')(x)# 32 4 512
    
    x=aa=Conv2D(512, (2, 2), strides=(1, 1), activation='relu', padding='valid', name='conv6_1')(x)# 32 4 512
    s=x.shape
    x=keras.layers.Reshape((s[1],s[3]),name='reshape_1')(x)#32 2048
    x=Bidirectional(LSTM(256, return_sequences=True,name='bidirectional_1'))(x)
    x=Bidirectional(LSTM(256, return_sequences=True,name='bidirectional_1'))(x)#32 512
    
    num_classes=87
    x=keras.layers.Dense(num_classes, name='dense1')(x)#알파벳+숫자#1953 36
    
    x=y_pred= Activation('softmax', name='softmax')(x)#1953 36
    
   # model_pred=keras.models.Model(inputs=[image_input,minx,miny,maxx,maxy],outputs=x)
    model_pred=keras.models.Model(inputs=image_input,outputs=x)
    
    maxstringlen=s[1]
    #maxstringlen=100
    
    def ctc_lambda_func(args):
        y_pred,labels, input_length, label_length = args
          #y_pred = y_pred[:, 2:, :] 
        return K.ctc_batch_cost(y_true=labels,y_pred=y_pred,input_length=input_length,label_length=label_length)    
        #return tf.nn.ctc_loss(labels=labels,logits=y_pred,logit_length=input_length,label_length=label_length,logits_time_major=False)    
    
    labels = Input(name='labels', shape=[maxstringlen], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64') 
    
    ctcloss=Lambda(ctc_lambda_func,output_shape=(1,),name='ctcloss')([y_pred,labels,input_length,label_length])
    
   # model_train=keras.models.Model(inputs=[image_input,minx,miny,maxx,maxy,labels,input_length,label_length],outputs=ctc_loss)
    model_train=keras.models.Model(inputs=[image_input,labels,input_length,label_length],outputs=ctcloss)
    
    return model_train,model_pred,maxstringlen

    

In [21]:
traincrnn,predcrnn,inputlength=crnn()
traincrnn.load_weights('checkpoints/202107191539_crnn_train_v1/weights.005.h5',by_name=True)
freeze = ['conv1_1',
          'conv2_1',
          'conv3_1', 
          'conv3_2', 
          'conv4_1',
          'conv5_1',
          #'conv6_1',
          #'lstm1',
          #'lstm2'
         ]
for layer in traincrnn.layers:
#    layer.trainable=not layer.name in freeze
    layer.trainable=True
traincrnn.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        [(None, 256, 32, 1)] 0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 256, 32, 64)  640         image_input[0][0]                
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 128, 16, 64)  0           conv1_1[0][0]                    
__________________________________________________________________________________________________
conv2_1 (Conv2D)                (None, 128, 16, 128) 73856       pool1[0][0]                      
____________________________________________________________________________________________

In [8]:
inputlength

62

In [14]:
croptrainpath='D:/engocr/cropvalimage/'
cropvalpath='D:/engocr/croptestimage/'
croptrainjson='D:/engocr/valcrnn.json'
cropvaljson='D:/engocr/traincrnn.json'

batch=64
traingen=crnnGenerator(imgpath=croptrainpath,labelpath=croptrainjson,imgw=256,imgh=32,batch_size=batch,
                      maxlen=62,inputlen=inputlength)
print('trainstart')
traingen.build_data()
print('valstart')
valgen=crnnGenerator(imgpath=cropvalpath,labelpath=cropvaljson,imgw=256,imgh=32,batch_size=batch,
                      maxlen=62,inputlen=inputlength)
valgen.build_data()
print('valend')

trainstart
valstart
valend


In [15]:
"""
import winsound as sd

def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()
"""
inputlength

62

In [22]:
traincrnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=0.001,decay=1e-5, clipnorm=1.) ,loss={'ctcloss':lambda y_true,y_pred:y_pred})

In [23]:
from tensorflow.python.keras.callbacks import TensorBoard  ## TensorBoard 를 import합니다.
modelver='crnn_train_v1'
checkdir = './checkpoints/' + datetime.datetime.now().strftime('%Y%m%d%H%M') + '_' + modelver
if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())
log_dir = "./logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard =TensorBoard(log_dir=log_dir,histogram_freq=1)

In [24]:
q=np.zeros((1,64,128))
len(q[0])

64

In [25]:
#[image_input,labels,input_length,label_length]
"""
his=traincrnn.fit(x=[valdata,valdata.labels,np.array( inputlengthlist),np.array(vallabellength)],
              y=valdata.labels,
                batch_size=16, 
              epochs=15,
              callbacks=[keras.callbacks.ModelCheckpoint(checkdir+'/weights.{epoch:03d}.h5', verbose=1, save_weights_only=True),
                         tensorboard]
             )
"""
his=traincrnn.fit_generator(generator=traingen.next_batch(),
                            steps_per_epoch=int(traingen.imgcount/batch),
                            epochs=30,
                            callbacks=[keras.callbacks.ModelCheckpoint(checkdir+'/weights.{epoch:03d}.h5', verbose=1, moniter='loss'),
                            tensorboard],
                            validation_data=valgen.next_batch(),
                            validation_steps=int(valgen.imgcount/batch)
                            
                           
                           
                           )



Epoch 1/30
1652/1652 [==============================] - 419s 250ms/step - loss: 7.0656 - val_loss: 5.7118

Epoch 00001: saving model to ./checkpoints/202107191719_crnn_train_v1\weights.001.h5
Epoch 2/30
1652/1652 [==============================] - 411s 249ms/step - loss: 5.1022 - val_loss: 5.3008

Epoch 00002: saving model to ./checkpoints/202107191719_crnn_train_v1\weights.002.h5
Epoch 3/30
1652/1652 [==============================] - 411s 249ms/step - loss: 4.4916 - val_loss: 5.0988

Epoch 00003: saving model to ./checkpoints/202107191719_crnn_train_v1\weights.003.h5
Epoch 4/30
1652/1652 [==============================] - 411s 249ms/step - loss: 4.0823 - val_loss: 5.1621

Epoch 00004: saving model to ./checkpoints/202107191719_crnn_train_v1\weights.004.h5
Epoch 5/30
1652/1652 [==============================] - 411s 249ms/step - loss: 3.7206 - val_loss: 5.1097

Epoch 00005: saving model to ./checkpoints/202107191719_crnn_train_v1\weights.005.h5
Epoch 6/30
1652/1652 [==================

KeyboardInterrupt: 

tf.Tensor(
[[[[ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   ...
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]]

  [[ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   ...
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]]

  [[ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   ...
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]]

  ...

  [[ 30.         30.         42.       ]
   [ 30.         30.         42.       ]
   [ 30.         30.         42.       ]
   ...
   [ 63.         62.         42.       ]
   [ 63.         62.         42.       ]
   [ 

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,


8001

[[28, 27, 28, 28, 26, 32, 27, 35, 34, 31],
 [17, 4, 3],
 [2, 14, 17, 13, 4, 17],
 [28, 28, 26],
 [2, 14, 17, 13, 4, 17],
 [17, 4, 3],
 [2, 0, 5, 4],
 [0, 19, 12],
 [28, 30],
 [7, 14, 20, 17],
 [0, 19, 12],
 [0],
 [19],
 [12],
 [18, 0, 11, 0, 3, 18],
 [2, 0, 5, 4],
 [2, 14, 17, 13, 4, 17],
 [17, 4, 3],
 [22, 14, 10],
 [4],
 [18, 0, 13, 3, 12, 8, 2, 7, 4, 18],
 [5, 14, 14, 3],
 [19, 0, 10, 4],
 [14, 20, 19],
 [1, 1],
 [2, 14, 5, 5, 4, 4],
 [1, 0, 17],
 [10, 4, 1, 0, 1, 18],
 [18, 12, 14, 14, 19, 7, 8, 4, 18],
 [2, 0, 5, 4],
 [8, 1, 12],
 [8, 1, 12],
 [13, 14, 22],
 [15, 11, 0, 24, 8, 13, 6],
 [8, 13],
 [19, 7, 4],
 [22, 8, 13, 13, 4, 17, 18],
 [2, 11, 14, 20, 3],
 [27, 27, 28, 26, 27, 26],
 [30, 28, 26, 27, 26],
 [27, 26, 28, 26, 26, 35],
 [17, 4, 18, 4, 0, 17, 2, 7],
 [31, 28, 26, 26, 35],
 [27, 28, 28, 26, 26, 34],
 [34, 28, 26, 26, 34],
 [31, 28, 26, 26, 34],
 [27, 26, 26],
 [32, 28, 26, 26, 33],
 [27, 27, 27, 28, 26, 27, 26],
 [35, 26],
 [34, 26],
 [33, 26],
 [3, 4, 4, 15, 16, 0],
 [